#### Calculate reference period percentiles of gridded soil moisture (sm) for each day of the year
#### These percentiles will then be used as thersholds to classify sm and analyse relationships between sm & P/P-E

In [1]:
%who

Interactive namespace is empty.


In [1]:
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=16,memory="31GB",walltime='02:30:00')
client = Client(cluster)
cluster.scale(cores=32)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.148:40659,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [2]:
import xarray as xr
import numpy as np
import climtas
    
out_dir = '/g/data/w97/ad9701/p_prob_analysis/temp_files/sm_refPeriod_perc/'
out_file = 'sm_191101_to_202005_30_40perc.nc'
# out_file = 'sm_191101_to_202005_perc.nc'
    
awra_dir = '/g/data/fj8/BoM/AWRA/DATA/SCHEDULED-V6/processed/values/day/'
sm_files = 'sm_[1-2]*.nc'

ds_sm_temp = xr.open_mfdataset(awra_dir + sm_files)
lat_slice = slice(-32, -39)      #slice(-36.3, -36.2)  #tiny slice for testing
lon_slice = slice(139, 152)      #slice(148.9, 149)
time_slice = slice('1911-01-01', '2020-05-31')

# converting the datatypes of SM to match P
lat_new = np.float32(ds_sm_temp['latitude'])
lon_new = np.float32(ds_sm_temp['longitude'])

# rename & reassign lat-lon to match the precip data; subset lat-lon
ds_sm = ds_sm_temp.rename({'latitude':'lat','longitude':'lon'}).assign_coords(lat=lat_new, lon=lon_new)\
.sel(lat = lat_slice, lon = lon_slice, time = time_slice)

# calculate percentiles using the full 31-day windows from all the available years
from tqdm.auto import tqdm
doy_pct = []
for doy, sample in tqdm(ds_sm.sm.rolling(time=31, center=True).construct(time='window').groupby('time.dayofyear')):
    # doy_pct.append(sample.load().quantile([0.1, 0.2, 0.3, 0.5], dim=['time', 'window']))
    doy_pct.append(sample.load().quantile([0.3, 0.4], dim=['time', 'window']))

da_sm_perc = xr.concat(doy_pct, dim='dayofyear')
dayofyear = np.arange(366)+1
da_sm_perc = da_sm_perc.assign_coords(dayofyear=dayofyear)

da_sm_perc.to_netcdf(out_dir + out_file)

cluster.scale(cores=0)

  0%|          | 0/366 [00:00<?, ?it/s]

/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.07/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1395: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


In [6]:
! squeue -u ad9701

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             11927       vdi sys/dash   ad9701  R    2:42:51      1 ood-vn17


In [ ]:
len(doy_pct)

In [5]:
cluster